In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import os
import random

import torch

In [ ]:
datapath =

# creating GPT-2 architechture

In [ ]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of transformer layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [ ]:
class LayerNorm(torch.nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = torch.nn.Parameter(torch.ones(emb_dim))
        self.shift = torch.nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [ ]:
class GELU(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))

In [ ]:
class FeedForward(torch.nn.Module):
    def __init__(self, config): ## config is a dictionary that contains the metadata of the gpt2 model
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(config["emb_dim"], 4 * config["emb_dim"]),
            GELU(),
            torch.nn.Linear(4 * config["emb_dim"], config["emb_dim"])
        )

    def forward(self, x):
      return self.layers(x)

In [ ]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, embedding_dim, context_length, num_heads,final_out_dim, dropout, qkv_bias=False):
        super().__init__()
        assert (final_out_dim % num_heads == 0), \
            "final_out_dim must be divisible by num_heads"

        self.final_out_dim =final_out_dim
        self.num_heads = num_heads
        self.head_dim =final_out_dim // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = torch.nn.Linear(embedding_dim,final_out_dim, bias=qkv_bias)
        self.W_key = torch.nn.Linear(embedding_dim,final_out_dim, bias=qkv_bias)
        self.W_value = torch.nn.Linear(embedding_dim,final_out_dim, bias=qkv_bias)

        self.out_proj = torch.nn.Linear(final_out_dim,final_out_dim)  # Linear layer to combine head outputs

        self.dropout = torch.nn.Dropout(dropout)

        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, embedding_dim = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens,final_out_dim)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens,final_out_dim) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        # attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = torch.softmax(attn_scores / self.head_dim**0.5, dim=-1)

        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.final_out_dim)

        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [ ]:
class TransformerBlock(torch.nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            embedding_dim = cfg["emb_dim"],
            final_out_dim = cfg["emb_dim"],
            context_length = cfg["context_length"],
            num_heads = cfg["n_heads"],
            dropout = cfg["drop_rate"],
            qkv_bias = cfg["qkv_bias"])

        self.ff = FeedForward(cfg)

        self.norm1 = LayerNorm(cfg["emb_dim"])

        self.norm2 = LayerNorm(cfg["emb_dim"])

        self.drop_shortcut = torch.nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        residual_conn1 = x
        x = self.norm1(x)
        x = self.att(x)  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + residual_conn1  # Add the original input back

        # Shortcut connection for feed forward block
        residual_conn_2 = x
        x = self.norm2(x)
        x = self.ff(x)
        # 2*4*768
        x = self.drop_shortcut(x)
        x = x + residual_conn_2  # Add the original input back

        return x
        # 2*4*768

In [ ]:
class GPT(torch.nn.Module):
  def __init__(self,cfg):
    super().__init__()
    self.token_emb = torch.nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
    self.position_emb = torch.nn.Embedding(cfg["context_length"], cfg["emb_dim"])

    self.drop_emb = torch.nn.Dropout(cfg["drop_rate"])

    self.blocks = torch.nn.Sequential(
        *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])]
    )

    self.final_norm = LayerNorm(cfg["emb_dim"])

    self.out_head = torch.nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

  def forward(self, x):
    batch_size, seq_len = x.shape
    token_emb = self.token_emb(x)
    pos_emb = self.position_emb(torch.arange(seq_len, device=x.device))

    x = token_emb + pos_emb
    x = self.drop_emb(x)

    x = self.blocks(x)
    x = self.final_norm(x)

    logits = self.out_head(x)

    return logits

# loading the pretrained GPT weights